# KMEANS

### 18/05/20

## Importing Libraries

In [6]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import sys
#sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#from AdaBandFlt import *
from V2AdaBandFlt import *

%matplotlib tk

## Loading Data

In [7]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbaseRaw.txt'


# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')
df

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,El 41,El 42,El 44,El 54,El 55,El 56,El 58,El 86,El 15,El 51
%t,,,,,,,,,,
0.00,0.92,-1.46,2.32,-0.61,0.06,-1.04,8.06,-0.24,0.73,-0.55
0.02,0.49,-1.53,1.59,-0.31,0.73,-1.22,6.29,-0.85,0.92,-1.40
0.04,0.43,-1.34,0.92,-0.37,1.16,-1.10,4.15,-0.92,0.85,-2.08
0.06,0.85,-0.85,0.37,-0.85,1.04,-0.92,2.08,-0.73,0.61,-2.44
0.08,1.46,-0.37,-0.12,-1.46,0.31,-0.73,0.55,-0.55,0.18,-2.38
...,...,...,...,...,...,...,...,...,...,...
299999.90,1.34,0.73,4.21,-1.95,-4.76,-1.28,-6.96,-3.60,-0.31,2.50
299999.92,1.59,0.49,4.09,-2.44,-3.42,-1.34,-6.04,-4.09,0.00,2.93
299999.94,2.08,0.37,3.66,-3.48,-1.95,-1.53,-5.13,-4.64,0.18,2.93


 # Filtering the signal for all electrodes

### Global Parameters

In [8]:
## Parameters to choose from: 

size_of_data = 1000000
fs = 50000

method_align = 'indice_1er_depass'
t_before = 0.0015
t_after = 0.0015

n_electrode = 3

### Filtering Functions

In [9]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

### Filtering the electrode signals...

In [26]:
lowcut = 100.0
highcut = 5000.0

filtered_signal = []

for i in range(n_electrode):
    before_filter = df.iloc[:size_of_data,i]
    
    y = butter_bandpass_filter(before_filter, lowcut, highcut, fs, order=6)
    
    filtered_signal.append(y)
    
filtereddf_not_transposed = pd.DataFrame(filtered_signal)

    
filtereddf = filtereddf_not_transposed.transpose()
filtereddf.index = df.index[:size_of_data]

KeyboardInterrupt: 

In [7]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - burst_data : part of signal where burst (after bandpass)


part_of_signal = signal_filtered

signal = part_of_signal

## Detecting and Aligning the Spikes

### Setting Up Thresholds adapted to Noise Levels

In [10]:

noise_levels = init_noise_levels(signal_filtered, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 20,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)


In [11]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

### Detecting Spikes

In [12]:
spike_info = find_spikes(signal, noise_levels, fs, 
                          window_size = 0.001, 
                          noise_window_size = 0.01,
                          threshold_factor = 3.0,
                          maxseparation = 0.002)

spike_info

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes
0,31218,31217,nan,31289,20.639037
1,48001,47999,nan,48092,19.683678
2,76238,76236,nan,76327,20.066465
3,166605,166603,nan,166694,20.352030
4,176846,176844,nan,176880,19.079949
...,...,...,...,...,...
81,2810820,2810819,nan,2810877,18.033982
82,2823673,2823671,nan,2823707,19.410650
83,2846907,2846907,nan,2846952,18.190087
84,2933931,2933929,nan,2933970,23.288531


### Recording the spikes

In [21]:
spike_data = record_spikes(signal, fs, spike_info,
                           method_align,
                           t_before = 0.0015, 
                           t_after = 0.0015)


(87, 151)


In [23]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_info,
                                           method_align,
                                           t_before = 0.0015,
                                           t_after = 0.0015)

### Plotting the spikes

In [25]:
#signal.plot(color = 'blue')
#spike_data_oneline.plot(color = 'red')
plt.plot(df.index[:size], signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)

No handles with labels found to put in legend.
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [27]:
print_spikes(spike_data,
             t_before_alignement = 0.0015,
             first_spike = 1,
             last_spike = 50,
             fs = fs)

# Bilan PCA + KMEANS

## PCA and KMEANS on spikes

In [28]:
def PCA_and_KMEANS_spikes(spike_data, spike_info, nb_PCA_components=3, nb_clusters=3):
    
    ## PCA
    pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
    pca = PCA(n_components=nb_PCA_components)
    pca.fit(pca_data)
    PCA_X = pca.transform(pca_data)
    
    ## KMEANS
    kmeans = KMeans(n_clusters=nb_clusters, random_state=0).fit(PCA_X)

    
    ## Ajout du label des clusters dans spike info
    spike_info['cluster_label'] = kmeans.labels_
    
    return PCA_X, kmeans, spike_info
    

## Plotting the PCA

In [29]:
## Fonction qui plot la PCA

def PCA_plot(PCA_X):
    
    fig = plt.figure(4,figsize=(4,3))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    plt.cla()
  
    ax.scatter(PCA_X[:, 0], PCA_X[:, 1], PCA_X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])

    plt.show()

In [30]:
## Importance des premiers coefficients de la PCA selon les différents indices d'alignement

def print_pca_explained_variance_ratio(signal, fs, spike_infos):
    legend = ['indice_max',
     'indice_min',
     'indice_depass_positif',
     'indice_depass_negatif',
     'indice_1er_depass',
     'indice_zero_central']
    explained_variance_ratio = []
    
    for method in legend:
        spike_data = record_spikes(signal, fs, spike_infos,
                  method,
                  t_before = 0.001,
                  t_after = 0.002)
        pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
        pca = PCA(n_components=10)
        pca.fit(pca_data)
        X = pca.transform(pca_data)
        explained_variance_ratio.append(pca.explained_variance_ratio_)
        
    plt.figure()
    plt.plot(np.transpose(explained_variance_ratio))
    plt.legend(legend)

## Plotting the KMEANS

In [31]:
## Fonction qui plot le KMEANS

def print_clusters_3d(labels, PCA_X):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    nb_clusters = max(labels) + 1

    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])

    for nb in range(nb_clusters):
        legend = 'Cluster n°'+str(nb)
        data = PCA_X[[x for x,y in b if y==nb],:]
        ax.scatter(data[:,0], data[:,1], data[:,2],label=legend)
    data = PCA_X[[x for x,y in b if y==-1],:]
    ax.scatter(data[:,0], data[:,1], data[:,2], c='black',label='Noise cluster')    
    
    ax.set_title('Nombre de cluster(s) :' + str(nb_clusters))
    ax.legend()
    plt.show()
    
#print_clusters_3d(kmeans.labels_, X)

## Plotting the spikes from the different clusters after KMEANS

In [32]:
## Fonction qui plot les spikes dans les clusters du KMEANS

def print_spikes_clusterized(spike_data,
                             labels,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000):
    
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 1
    else:
        nb_clusters_ = nb_clusters
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    #spike_data.iloc[:,first_spike:last_spike].plot()
    t_b = int(np.round(fs*(t_before_alignement)))
    y = (spike_data.iloc[:,0]-t_b)*1000/fs
        
    a = [i+1 for i in range(len(labels))]
    b = np.transpose([a,list(labels)])
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        data = spike_data.iloc[:,[x for x,y in b if y==nb]]
        m = len(data.values[0])
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster numero ' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        
    if (-1 in labels) == True:
        data = spike_data.iloc[:,[x for x,y in b if y==-1]]
        m = len(data.values[0])
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        
#print_spikes_clusterized(spike_data,
#                             dbscan.labels_,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)
        

#print_spikes_clusterized(spike_data,
#                             kmeans.labels_,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)

## Printing the spikes from clusters on original signal

In [33]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline(signal,
                                     updated_spike_infos,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):
    
    labels = updated_spike_infos['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 2
    else:
        nb_clusters_ = nb_clusters + 1
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    ghost_array = np.array(['NaN' for x in range(len(signal))])
    ghost_array = ghost_array.astype(float)
    spike_data_clusterized_oneline = []
    legend = ['Signald\'origine']
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == nb],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(signal.index, signal.values)
        for ghost in range(nb):
            axes.plot(signal.index, ghost_array)
        axes.plot(signal.index, spike_data_oneline)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster n°' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
        
        legend.append('Cluster n°' + str(nb))
        
    if (-1 in labels) == True:
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == -1],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(signal.index, signal.values)
        axes.plot(signal.index, spike_data_oneline)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
                
        legend.append('Cluster de bruit')
        
        axes = figure.add_subplot(nb_line, 2, nb+3)
    else:
        axes = figure.add_subplot(nb_line, 2, nb+2)
    
    axes.plot(signal.index, signal.values)
    axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
    axes.legend(legend)
    axes.set_xlabel('Time in ms')
    axes.set_title('Tous les clusters')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

In [34]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline_total(signal,
                                     updated_spike_infos,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):
    
    labels = updated_spike_infos['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 2
    else:
        nb_clusters_ = nb_clusters + 1
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    ghost_array = np.array(['NaN' for x in range(len(signal))])
    ghost_array = ghost_array.astype(float)
    spike_data_clusterized_oneline = []
    legend = ['Signald\'origine']
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    for nb in range(nb_clusters):
        
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == nb],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        
        legend.append('Cluster n°' + str(nb))

        
    if (-1 in labels) == True:
        spike_data_oneline = record_spikes_oneline(signal,
                              fs,
                              updated_spike_infos.loc[updated_spike_infos['cluster_label'] == -1],
                              align_method,
                              t_before = t_before,
                              t_after = t_after)
        spike_data_oneline = np.resize(spike_data_oneline.values,len(spike_data_oneline.values))
        spike_data_clusterized_oneline.append(spike_data_oneline)
        legend.append('Cluster de bruit')
    
    axes = figure.add_subplot(1, 1, 1)
    axes.plot(signal.index, signal.values)
    axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
    axes.legend(legend)
    axes.set_xlabel('Time in ms')
    axes.set_title('Tous les clusters')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline_total(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

## Histogram of Amplitudes and Time

In [35]:
## Histogramme de différences d'amplitude Peak-to-Peak des spikes

def histogram_spikes_amplitude(spike_info):
    plt.figure()
    plt.hist(spike_info['Delta_amplitudes'], bins='auto')
    plt.title("Histogram of Spike Peak-to-Peak Amplitude")
    plt.show()

In [36]:
## Histogramme de l'écartement temporel des spikes (imax-imin)

def histogram_spikes_time(spike_info):
    plt.figure()
    plt.hist(spike_info['i_max-i_min'], bins='auto')
    plt.title("Histogram of Spike i_max-i_min")
    plt.show()

In [37]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized(updated_spike_info):
    
    labels = updated_spike_info['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters):
        local_info = spike_info.loc[spike_info['cluster_label'] == nb]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster n°"+str(nb))
        plt.show()
    
    if(-1 in labels) == True:
        local_info = spike_info.loc[spike_info['cluster_label']==-1]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster of noise")
        plt.show()

In [38]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized_superposed(updated_spike_info):
    
    labels = updated_spike_info['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters,0,-1):
        local_info = spike_info.loc[spike_info['cluster_label'] == nb]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Superposition of histogram of Spike peak-to-peak of clusters")
        plt.show()
    
    if(-1 in labels) == True:
        local_info = spike_info.loc[spike_info['cluster_label']==-1]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.show()

# Tests des fonctions

In [40]:
# Function: PCA_and_KMEANS_spikes(spike_data, spike_info, nb_PCA_components=3, nb_clusters=3)

PCA_X, KMEANS, updated_spike_info = PCA_and_KMEANS_spikes(spike_data, spike_info, 5, 6)

In [41]:
# Function: PCA_plot(spike_data, nb_clusters=3)

PCA_plot(PCA_X)

In [42]:
# Function: print_clusters_3d(labels, PCA_X)

print_clusters_3d(KMEANS.labels_, PCA_X)

In [43]:
# Function: print_spikes_clusterized(spike_data,
#                             labels,
#                             t_before_alignement = 0,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)
                        
print_spikes_clusterized(spike_data,
                             KMEANS.labels_,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = fs)                        

In [45]:
# Function: print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

print_spikes_clusterized_oneline(signal,
                                 updated_spike_info,
                                 align_method = method_align,
                                 t_before = 0.0015,
                                 t_after = 0.0015,
                                 fs = fs,)

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [49]:
print_spikes_clusterized_oneline_total(signal,
                                 updated_spike_info,
                                 align_method = method_align,
                                 t_before = 0.0015,
                                 t_after = 0.0015,
                                 fs = fs,)

In [ ]:
# Function: histogram_spikes_amplitude(spike_info)

histogram_spikes_amplitude(spike_info)

In [ ]:
# Function: histogram_spikes_time(spike_info)

histogram_spikes_time(spike_info)

In [ ]:
# Function: histogram_amplitude_clusterized(spike_info)

histogram_amplitude_clusterized(updated_spike_info)

In [ ]:
# Function: histogram_amplitude_clusterized_superposed(updated_spike_info)

histogram_amplitude_clusterized_superposed(updated_spike_info)

In [65]:
print_pca_explained_variance_ratio(signal, fs, spike_info)

(38, 151)
(38, 151)
(38, 151)
(38, 151)
(38, 151)
(38, 151)
